In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from dotenv import load_dotenv
import os

import pandas as pd

from collections import defaultdict

# ---<---<--<--@    hasta aquí nuestras librerías   @-->-->--->---

# Identifica nuestras claves guardadas con dotenv
#load_dotenv() 
# client_id = os.getenv("CLIENT_ID")
# client_secret = os.getenv("CLIENT_SECRET")
client_id_spotify_micaela = "10be6d86e56744838061c07f2b48f2f2"
client_secret_spotify_micaela = "80b4582ce5b542f1978e7fbf785789db"

# Autenticación API Spotify
auth_manager = SpotifyClientCredentials(client_id=client_id_spotify_micaela, client_secret=client_secret_spotify_micaela)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Función para automatizar la extracción de lo que queremos solicitar:
# -> artista, género, canción/album, año de lanzamiento, id_lanzamiento

def extraer_datos_por_genero_y_rango(genero, rango_anyo, resultados_por_anyo=200):
    albumes_por_anyo = defaultdict(list)
    canciones_por_anyo = defaultdict(list)

    for o in range(0, 200, 50):
        artistas = sp.search(q=f'genre:{genero}', type='artist', limit=50, offset=o)
        for artista in artistas['artists']['items']:
            artista_id = artista['id']
            artista_nombre = artista['name']

            # aquí extraemos álbumes
            albumes_data = sp.artist_albums(artista_id, album_type=['album'], limit=50)
            for album in albumes_data['items']:
                try:
                    album_lanzamiento = int(album['release_date'][:4])
                    if album_lanzamiento in rango_anyo and len(albumes_por_anyo[album_lanzamiento]) < resultados_por_anyo:
                        albumes_por_anyo[album_lanzamiento].append({
                            'artista': artista_nombre,
                            'genero': genero,
                            'tipo': album['album_type'],
                            'nombre': album['name'],
                            'año': album_lanzamiento,
                            'id': album['id']
                        })
                except (KeyError, ValueError, TypeError):
                    continue

            # aquí extraemos canciones
            try:
                top_tracks = sp.artist_top_tracks(artista_id)
                for track in top_tracks['tracks']:
                    try:
                        track_lanzamiento = int(track['album']['release_date'][:4])
                        if track_lanzamiento in rango_anyo and len(canciones_por_anyo[track_lanzamiento]) < resultados_por_anyo:
                            canciones_por_anyo[track_lanzamiento].append({
                                'artista': artista_nombre,
                                'genero': genero,
                                'tipo': 'cancion',
                                'nombre': track['name'],
                                'año': track_lanzamiento,
                                'id': track['id']
                            })
                    except (KeyError, ValueError, TypeError):
                        continue
            except 'SpotifyException':
                continue

    # devuelve un diccionario con dos listas separadas
    return {
        'albumes': [{'tipo': 'album', **data} for album_lanzamiento in rango_anyo for data in albumes_por_anyo[album_lanzamiento][:resultados_por_anyo]],
        'canciones': [{'tipo': 'cancion', **data} for anyo in rango_anyo for data in canciones_por_anyo[anyo][:resultados_por_anyo]]
    }



In [2]:
# Uso: 

tastyinfo_2018_2022 = extraer_datos_por_genero_y_rango('r&b', range(2018, 2023))
print(f'\nLlamada a Spotify (( ☎ )) Spotify ha respondido\n    . . . . . . . . . . . . . . . . . . .\n\nHemos obtenido {len(tastyinfo_2018_2022)} listas ☑\n')

albumes_2018_2022 = tastyinfo_2018_2022['albumes']  # separar una lista de álbumes
print(f'Se ha creado una lista con la info de los {len(albumes_2018_2022)} álbumes ☑')
canciones_2018_2022 = tastyinfo_2018_2022['canciones']  # separar una lista de canciones
print(f'Se ha creado una lista con la info de las {len(canciones_2018_2022)} canciones ☑\n')


# Conversión a Data Frame:

df_2018_2022_album = pd.DataFrame(albumes_2018_2022)
print(f'DataFrame álbumes del género {albumes_2018_2022[0]["genero"]} ☑\n')
df_2018_2022_tracks = pd.DataFrame(canciones_2018_2022)
print(f'DataFrame top tracks del género {albumes_2018_2022[1]["genero"]} ☑\n')

# Paso de exportación (de DataFrame a .csv):
# 
# Cada una tenemos que modificar el nombre que queremos para el archivo según su género 

#                         ---<-----<-----@ archivo para albumes a .CSV @----->---->---

df_2018_2022_album.to_csv('rock_2018_2022_albumes.csv', index=False, encoding='utf-8')
print("""
ALBUMES                 2018                                           
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022               
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--                      --====--         ---======--- \n""")

#                       --<-----<-----@ archivo para canciones a .CSV @----->---->---

df_2018_2022_tracks.to_csv('rock_2018_2022_canciones.csv', index=False, encoding='utf-8')
print("""
TOP TRACKS              2018                                              
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022              
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--                      --====--         ---======--- \n""")




Llamada a Spotify (( ☎ )) Spotify ha respondido
    . . . . . . . . . . . . . . . . . . .

Hemos obtenido 2 listas ☑

Se ha creado una lista con la info de los 382 álbumes ☑
Se ha creado una lista con la info de las 425 canciones ☑

DataFrame álbumes del género r&b ☑

DataFrame top tracks del género r&b ☑


ALBUMES                 2018                                           
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022               
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--                     